In [ ]:
indir =  "/data2/mito_lineage/output/pipeline/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_mgatkdonor/vireo/nclones20/enrichment/annotation_clones/DE_large"
outdir = "/data2/mito_lineage/output/pipeline/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_mgatkdonor/vireo/nclones20/enrichment/annotation_clones/DE_large/minPct_0.01__logfcthresh_0"
n_donors = 4
# se_f = "/data2/mito_lineage/output/annotation_clones/data/cd34norm/MTblacklist/SE.rds"
# outdir = "/data2/mito_lineage/output/annotation_clones/data/cd34norm/MTblacklist/DE_large"

# If more than 2 clones pass the cdf_thresh, use that, o.w use n_top_clones
cdf_thresh = 0.5
n_top_clones = 3

min_pct = 0.01
logfc_threshold = 0.1
use_latent_vars = FALSE
genome = "/data2/mito_lineage/data/external/GRCh38_MT_blacklist/fasta/genome.fa"
cores=16
utils_script <- "/data2/mito_lineage/R_scripts/seurat_utils.R"

In [ ]:
if (use_latent_vars){
    latent.vars = "peaks_depth"
}else{
    latent.vars=NULL
}

In [ ]:
source(utils_script)

library(Seurat)
library(Signac)
library(ggplot2)
library(patchwork)
set.seed(1234)
library(magrittr)
library(cowplot)
library(metap)
library(dplyr)
library(future)
plan("multiprocess", workers = cores)
#options(future.globals.maxSize = 50000 * 1024^2) # for 50 Gb RAM
options(future.globals.maxSize = 8000 * 1024^2)
library("ComplexHeatmap")

## Import seurat object and clones

In [ ]:
large.clones <- read.csv(file=file.path(indir, "large_clones.csv"))
se <- readRDS(file=file.path(indir, "se.clonesfilt.rds"))    

## Run DE

In [ ]:
for (d in 0:(n_donors-1)){
    print('donor')
    print(d)
    curr.outdir <- file.path(outdir, paste0("donor", d, "_TF"))
    dir.create(curr.outdir)
    donor.large.clones <- large.clones %>% filter(donor==d)
    clones.filt.ids <- sort(donor.large.clones$lineage)
    print('clones')
    print(clones.filt.ids)
    print(length(clones.filt.ids))
    se.filt <- subset(se, subset = (donor==d) & (lineage %in% donor.large.clones$lineage))
    Idents(se.filt) <- se.filt$lineage  
    print('se.filt')
    print(dim(se.filt[[]]))

    pairs = combn(clones.filt.ids,2)
    for (i in 1:dim(combn(clones.filt.ids,2))[2]){
        DefaultAssay(se.filt) <- "chromvarnames"
        print(pairs[,i])
        try({
            print(paste("clones", i))
            a = pairs[1,i]
            b = pairs[2,i]
            da <- FindMarkers(
              object = se.filt,
              ident.1 = a,
              ident.2 = b,
              only.pos = FALSE,
              mean.fxn = rowMeans,
              logfc.threshold = logfc_thresh,
              min.pct = min_pct,
              #latent.vars=latent.vars,
              fc.name = "avg_diff")
            
            da$p_val_adj_BH <- stats::p.adjust(da$p_val, method = "BH", n = length(da$p_val))
            write.csv(da, 
                      file=file.path(curr.outdir, paste0("clones_",a,"__", b,".DE.TF.csv")), quote=F)
            #plotDE(se, da, c, clust_outdir)
    })
            gally <- GGally::ggpairs(da[,c("p_val", "p_val_adj", "avg_diff", "p_val_adj_BH" )], aes(alpha = 0.4))
            ggsave(plot=gally, file=file.path(curr.outdir, paste0("clones_",a,"__", b,".DE.TF.pvalHist.png")))
            # Save motifs
            DefaultAssay(se.filt) <- "ATAC"
            raw.ids <- names(Motifs(se.filt)@motif.names)
            print(head(rownames(da)))
            rownames(da) <- sapply((rownames(da)), function(x){raw.ids[match(x, Motifs(se.filt)@motif.names)]})
            m <- MotifPlot(
              object = se.filt,
              motifs = head(rownames(da)),
              assay = 'ATAC'
            )
            ggsave(plot=m, 
                       file=file.path(curr.outdir, paste0("clones_",a,"__", b,".DE.TF.motif.top.png")))


        }
    }